In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt


In [4]:
def extract_answer(text):
    pattern = r"(True|False|Don't know)"
    matches = re.findall(pattern, text)
    return matches


In [43]:
RESPONSE_PATH = "../../../data/analysis_results/Llama_2_13b_chat_hf_prompt_based_binary_response.csv"

RESPONSE_PATH = "../../../data/analysis_results/Llama_2_13b_chat_hf_node_retrieval_rag_based_binary_response.csv"


In [44]:
response_df = pd.read_csv(RESPONSE_PATH)

response_df.loc[:, 'extracted_answer'] = response_df['llm_answer'].apply(extract_answer)


In [45]:
response_df.loc[:, "answer_count"] = response_df.extracted_answer.apply(lambda x:len(x))

response_df_multiple_answers = response_df[response_df.answer_count > 1]
response_df_single_answer = response_df.drop(response_df_multiple_answers.index)
response_df_single_answer.drop("answer_count", axis=1, inplace=True)


response_df_multiple_answers_ = []
for index, row in response_df_multiple_answers.iterrows():
    if row["extracted_answer"][0] == row["extracted_answer"][1]:
        response_df_multiple_answers_.append((row["question"], row["label"], row["llm_answer"], row["extracted_answer"][0]))
    else:
        response_df_multiple_answers_.append((row["question"], row["label"], row["llm_answer"], None))

response_df_multiple_answers_ = pd.DataFrame(response_df_multiple_answers_, columns=["question", "label", "llm_answer", "extracted_answer"])

response_df_final = pd.concat([response_df_single_answer, response_df_multiple_answers_], ignore_index=True)
response_df_final = response_df_final.explode("extracted_answer")



In [46]:
incorrect_answers_because_of_na = response_df_final[response_df_final.extracted_answer.isna()]


In [47]:
response_transform = {
    "True" : True,
    "False" : False
}

response_df_final.extracted_answer = response_df_final.extracted_answer.apply(lambda x:response_transform[x])



In [48]:
correct_response = response_df_final[response_df_final.label == response_df_final.extracted_answer]
incorrect_response = response_df_final[response_df_final.label != response_df_final.extracted_answer]



In [49]:
total_response = response_df_final.shape[0]
correct_response_count = correct_response.shape[0]
incorrect_response_count = incorrect_response.shape[0] + incorrect_answers_because_of_na.shape[0]

In [51]:
correct_response_ = correct_response_count/total_response
incorrect_response_ = incorrect_response_count/total_response


print("Correct response = ",correct_response_)
print("Incorrect response = ",incorrect_response_)



Correct response =  0.8850574712643678
Incorrect response =  0.11494252873563218


In [57]:
correct_response.llm_answer.values[0]

'  {\n"answer": "False"\n}\n\nExplanation:\n\nEnhanced S-cone syndrome is a retinal disease that presents with symptoms such as vision disorders, pigmentary retinopathy, retinoschisis, abnormality of the lens, macular edema, and cataract. It is not a vitreoretinal degeneration, as it does not involve the vitreous or retinal degeneration. Instead, it is a disease that affects the S-cones in the retina, leading to the symptoms mentioned above.'

In [58]:
correct_response.question.values[0]

'enhanced S-cone syndrome is not a vitreoretinal degeneration'

In [59]:
correct_response.shape

(539, 4)